In [1]:
#****************************************************************************
# (C) Cloudera, Inc. 2020-2024
#  All rights reserved.
#
#  Applicable Open Source License: GNU Affero General Public License v3.0
#
#  NOTE: Cloudera open source products are modular software products
#  made up of hundreds of individual components, each of which was
#  individually copyrighted.  Each Cloudera open source product is a
#  collective work under U.S. Copyright Law. Your license to use the
#  collective work is as provided in your written agreement with
#  Cloudera.  Used apart from the collective work, this file is
#  licensed for your use pursuant to the open source license
#  identified above.
#
#  This code is provided to you pursuant a written agreement with
#  (i) Cloudera, Inc. or (ii) a third-party authorized to distribute
#  this code. If you do not have a written agreement with Cloudera nor
#  with an authorized and properly licensed third party, you do not
#  have any rights to access nor to use this code.
#
#  Absent a written agreement with Cloudera, Inc. (“Cloudera”) to the
#  contrary, A) CLOUDERA PROVIDES THIS CODE TO YOU WITHOUT WARRANTIES OF ANY
#  KIND; (B) CLOUDERA DISCLAIMS ANY AND ALL EXPRESS AND IMPLIED
#  WARRANTIES WITH RESPECT TO THIS CODE, INCLUDING BUT NOT LIMITED TO
#  IMPLIED WARRANTIES OF TITLE, NON-INFRINGEMENT, MERCHANTABILITY AND
#  FITNESS FOR A PARTICULAR PURPOSE; (C) CLOUDERA IS NOT LIABLE TO YOU,
#  AND WILL NOT DEFEND, INDEMNIFY, NOR HOLD YOU HARMLESS FOR ANY CLAIMS
#  ARISING FROM OR RELATED TO THE CODE; AND (D)WITH RESPECT TO YOUR EXERCISE
#  OF ANY RIGHTS GRANTED TO YOU FOR THE CODE, CLOUDERA IS NOT LIABLE FOR ANY
#  DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, PUNITIVE OR
#  CONSEQUENTIAL DAMAGES INCLUDING, BUT NOT LIMITED TO, DAMAGES
#  RELATED TO LOST REVENUE, LOST PROFITS, LOSS OF INCOME, LOSS OF
#  BUSINESS ADVANTAGE OR UNAVAILABILITY, OR LOSS OR CORRUPTION OF
#  DATA.
#
# #  Author(s): Paul de Fusco
#***************************************************************************/

In [2]:
import pandas as pd
import geopandas
import folium
import matplotlib.pyplot as plt
import os, warnings, sys, logging
import pandas as pd
import numpy as np
from datetime import date
import cml.data_v1 as cmldata
import pyspark.pandas as ps
import seaborn as sns
import stumpy

In [3]:
USERNAME = os.environ["PROJECT_OWNER"]
DBNAME = "trafico_{}".format(USERNAME)
STORAGE = "s3a://col1-buk-ea8d2b79/data/"
CONNECTION_NAME = "col1-aw-dl"
DATE = date.today()

conn = cmldata.get_connection(CONNECTION_NAME)
spark = conn.get_spark_session()

24/04/15 09:19:31 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
24/04/15 09:19:31 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
Setting spark.hadoop.yarn.resourcemanager.principal to pauldefusco
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/15 09:19:32 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
24/04/15 

In [4]:
df_from_sql = ps.read_table('{0}.trafico_{1}'.format(DBNAME, USERNAME))
df = df_from_sql.to_pandas()

24/04/15 09:19:40 WARN HiveClientImpl: Detected HiveConf hive.execution.engine is 'tez' and will be reset to 'mr' to disable useless hive logic
Hive Session ID = bd25e60a-2e17-40ab-9f94-2158abd810eb
24/04/15 09:19:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/15 09:19:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:194: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


In [5]:
df['vehicles'] = df['vehicles'].astype("float64")
df['motorcycles'] = df['motorcycles'].astype("float64")
df['pedestrians'] = df['pedestrians'].astype("float64")
df['trucks'] = df['trucks'].astype("float64")

In [6]:
# Create point geometries
geometry = geopandas.points_from_xy(df.longitude, df.latitude)
geo_df = geopandas.GeoDataFrame(
    df[["id", "device_id", "event_ts", \
        "vehicles", "motorcycles", "pedestrians", "trucks", "traffic_condition"]], geometry=geometry
)

geo_df.head()

,id,device_id,event_ts,vehicles,motorcycles,pedestrians,trucks,traffic_condition,geometry
0,0,0x1000000000005,2023-12-01 01:00:00,50.0,8.0,219.0,52.0,low traffic,POINT (-75.13834 6.21985)
1,1,0x100000000001d,2023-12-01 01:01:00,28.0,5.0,204.0,28.0,high congestion,POINT (-74.73826 6.76978)
2,2,0x1000000000008,2023-12-01 01:02:00,31.0,10.0,261.0,32.0,average traffic,POINT (-75.30926 6.79754)
3,3,0x100000000001b,2023-12-01 01:03:00,10.0,5.0,294.0,12.0,low traffic,POINT (-74.58320 6.25749)
4,4,0x1000000000014,2023-12-01 01:04:00,32.0,1.0,210.0,32.0,average traffic,POINT (-75.16383 6.21757)


In [7]:
# OpenStreetMap
map = folium.Map(location=[6.24560, -76.56290], tiles="OpenStreetMap", zoom_start=9)

In [8]:
# Create a geometry list from the GeoDataFrame
geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in geo_df.geometry]

In [9]:
# Iterate through list and add a marker for each volcano, color-coded by its type.
i = 0
for coordinates in geo_df_list:
    # assign a color marker for the type of volcano, Strato being the most common
    if geo_df.traffic_condition[i] == "clear":
        type_color = "blue"
    elif geo_df.traffic_condition[i] == "low traffic":
        type_color = "green"
    elif geo_df.traffic_condition[i] == "average traffic":
        type_color = "orange"
    elif geo_df.traffic_condition[i] == "high congestion":
        type_color = "purple"
    else:
        type_color = "pink"
        
    # Place the markers with the popup labels and data
    map.add_child(
        folium.Marker(
            location=coordinates,
            popup="device_id: "
            + str(geo_df.device_id[i])
            + "<br>"
            + "motorcycles: "
            + str(geo_df.motorcycles[i])
            + "<br>"
            + "vehicles: "
            + str(geo_df.vehicles[i])
            + "<br>"
            + "pedestrians: "
            + str(geo_df.pedestrians[i])
            + "<br>"
            + "trucks: "
            + str(geo_df.trucks[i]),
            icon=folium.Icon(color="%s" % type_color),
        )
    )
    i = i + 1


In [10]:
map